# Loading the Saved Model and Checking Prediction Accuracy

In [8]:
!pip install joblib

## We predict flair for https://www.reddit.com/r/india/comments/g8r2pe/skylines_of_mussoorieuttarakhand/

In [9]:
import pickle
import logging
import gensim
import praw
from praw.models import MoreComments
import os
import json
import joblib
from gensim import utils
import gensim.parsing.preprocessing as gsp

filters = [
           gsp.strip_tags, 
           gsp.strip_punctuation,
           gsp.strip_multiple_whitespaces,
           gsp.strip_numeric,
           gsp.remove_stopwords, 
           gsp.strip_short, 
           gsp.stem_text
          ]

def clean(s):
    s = s.lower()
    s = utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return s

# model = bst


reddit = praw.Reddit(client_id = "F4OMSR9jAi_kFQ",
                    client_secret = "dBKDarfWBg7QyeZ-CXlNeLYlQ9A",
                    user_agent = "web_app:reddit_scraper:v1.1.1 (by u/skadwhoosh)",
                    username = "skadwhoosh",
                    password = "reddit@123")

model = joblib.load("../app/model/LR.pkl")
# result = loaded_model.score(X_test, Y_test)
# print(result)


def prediction(url):
    submission = reddit.submission(url = url)
    data = {}
    data["title"] = str(submission.title)
    data["url"] = str(submission.url)
    data["body"] = str(submission.selftext)

    submission.comments.replace_more(limit=None)
    comment = ''
    count = 0
    for top_level_comment in submission.comments:
        comment = comment + ' ' + top_level_comment.body
        count+=1
        if(count > 10):
            break

    data["comment"] = str(comment)

    data['title'] = clean(str(data['title']))
    data['body'] = clean(str(data['body']))
    data['comment'] = clean(str(data['comment']))
    
    combined_features = data["title"] + data["comment"] + data["body"] + data["url"]

    return str(model.predict([combined_features]))[2:-2]
 
prediction("https://www.reddit.com/r/india/comments/g8r2pe/skylines_of_mussoorieuttarakhand/")

'Photography'

## As we can see, the model groups it under 'Photography' flair which is fairly accurate according to the original post.